# SETUP

In [ ]:
!pip install -r requirements_kaggle.txt -q

> To speed up the review process , i provided the ***drive id*** of the data i've created from the Train creation folder noteboooks .
---
> I  also add each data drive link in the Readme Pdf file attached with this solution

In [ ]:
!pip install -q gdown

In [ ]:
!gdown --id 1hNRbtcqd9F6stMOK1xAZApDITwAjiSDJ
!gdown --id 1-QCmWsNGREXuWArifN0nD_Sp4hJxf0tu

In [ ]:
!gdown --id 1-47L_1NKLeVgW1vWmqXXXCuWZ3gwZWsS
!gdown --id 1-aO4FEtv5CF-ZOcxDSO3jGEzPcIFdxgP

In [ ]:
!gdown --id 1-8J_xFgI0WKT5UXFnfH4q1KUw_KgNY37
!gdown --id 1-a55a7N6a4SoqolPF_wI4C6Q70u_d7Hj

In [ ]:
!gdown --id 1-BgXQwmXqBuk_P8VtvLfdLqy83dv56Kz
!gdown --id 1-hQGF2TNBbsy3jsGNtndmK55egbdFDjs

# LIBRARIES

In [ ]:
#import necessary dependecies
import os
import numpy as np  
import pandas as pd

import random
from tqdm import tqdm 
import copy

from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.preprocessing import QuantileTransformer

import warnings
warnings.filterwarnings('ignore') 

# tf 
import tensorflow as tf
from tensorflow.keras.layers import Concatenate , Input ,concatenate ,add
from tensorflow.keras.models import Sequential , Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout
from tensorflow.keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from tensorflow.keras.preprocessing import sequence, text
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.initializers import glorot_normal

# fix seed
tf.random.set_seed(111)
np.random.seed(111)
random.seed(111)

## Train Creation

In [ ]:
def create_train():
  train =pd.read_csv("S2TrainObs1.csv" )

  train = train.groupby('field_id').median().reset_index().sort_values('field_id')
  train.label = train.label.astype('int')
  return train

In [ ]:
def create_test():
  test =pd.read_csv("S2TestObs1.csv" )
  return  test

In [ ]:
def createObs2_train():
  train =pd.read_csv("S2TrainObs2.csv" )

  train = train.groupby('field_id').median().reset_index().sort_values('field_id')
  train.label = train.label.astype('int')
  return train

In [ ]:
def createObs2_test():
  test =pd.read_csv("S2TestObs2.csv" )
  return test

In [ ]:
def createObs3_train():
  train =pd.read_csv("S2TrainObs3.csv" )

  train = train.groupby('field_id').median().reset_index().sort_values('field_id')
  train.label = train.label.astype('int')
  return train

In [ ]:
def createObs3_test():
  test =pd.read_csv("S2TestObs3.csv" )
  return test

In [ ]:
def createObs4_train():
  train =pd.read_csv("S2TrainObs4.csv" )

  train = train.groupby('field_id').median().reset_index().sort_values('field_id')
  train.label = train.label.astype('int')
  return train

In [ ]:
def createObs4_test():
  test =pd.read_csv("S2TestObs4.csv" )
  return test

# Feature Engineering

In [ ]:
def process(T) :

  # process bands
  Bcols = T.filter(like='B').columns.tolist()
  Vcols = T.filter(like='V').columns.tolist()
  Obs1 = T.filter(like='Month4').columns.tolist()
  Obs2 = T.filter(like='Month5').columns.tolist()
  Obs3 = T.filter(like='Month6').columns.tolist()
  Obs4 = T.filter(like='Month7').columns.tolist()
  Obs5 = T.filter(like='Month8').columns.tolist()
  Obs6 = T.filter(like='Month9').columns.tolist()
  Obs7 = T.filter(like='Month10').columns.tolist()
  Obs8 = T.filter(like='Month11').columns.tolist()


  # vegetation indexes 
  B8cols = T.filter(like='B8_').columns.tolist()
  B8cols = [x for x in B8cols if 'std' not in x]
  
  B4cols = T.filter(like='B4_').columns.tolist()
  B4cols = [x for x in B4cols if 'std' not in x]

  B3cols = T.filter(like='B3_').columns.tolist()
  B3cols = [x for x in B3cols if 'std' not in x]

  B5cols = T.filter(like='B5_').columns.tolist()
  B5cols = [x for x in B5cols if 'std' not in x]

  B3cols = T.filter(like='B3_').columns.tolist()
  B3cols = [x for x in B3cols if 'std' not in x]

  B2cols = T.filter(like='B2_').columns.tolist()
  B2cols = [x for x in B2cols if 'std' not in x]

  B7cols = T.filter(like='B7_').columns.tolist()
  B7cols = [x for x in B7cols if 'std' not in x]

  B8Acols = T.filter(like='B8A_').columns.tolist()
  B8Acols = [x for x in B8Acols if 'std' not in x]

  B6cols = T.filter(like='B6_').columns.tolist()
  B6cols = [x for x in B6cols if 'std' not in x]
  
  B12cols = T.filter(like='B12_').columns.tolist()
  B12cols = [x for x in B12cols if 'std' not in x]

  B11cols = T.filter(like='B11_').columns.tolist()
  B11cols = [x for x in B11cols if 'std' not in x]

  B1cols = T.filter(like='B1_').columns.tolist()
  B1cols = [x for x in B1cols if 'std' not in x]

  B9cols = T.filter(like='B9_').columns.tolist()
  B9cols = [x for x in B9cols if 'std' not in x]

  L = 0.725
  for b1,b2 ,b3 ,b4, b5 , b6, b7, b8 ,b8a ,b9,b11,b12 in zip(B1cols,B2cols,B3cols,B4cols,B5cols,B6cols,B7cols,B8cols,B8Acols,B9cols,B11cols,B12cols) :
    T[f'NDVI_{b8.split("_")[1]}']   = ((T[b8] - T[b4]) /  (T[b8] + T[b4])).values.clip(-5,5)
    T[f'SAVI_{b8.split("_")[1]}']   = ((T[b8] - T[b4]) /  (T[b8] + T[b4]+L) * (1.0 + L)).values.clip(-5,5)
    # T[f'CCCI_{b8.split("_")[1]}']   =  ((T[b8] - T[b5]) /  (T[b8] + T[b5])) / ((T[b8] - T[b4]) /  (T[b8] + T[b4]))
    T[f'GRNDVI_{b8.split("_")[1]}'] = ((T[b8] - (T[b3]+T[b4])) /  (T[b8] + (T[b3]+T[b4]))).values.clip(-5,5)
    T[f'GNDVI_{b8.split("_")[1]}']  = ((T[b8] - T[b3] ) /  (T[b8] + T[b3])).values.clip(-5,5)
    # T[f'WDRVI_{b8.split("_")[1]}']  = (0.1 * T[b5] - T[b4])/ (0.1 * T[b5] + T[b4])
    T[f'NDRE_{b8.split("_")[1]}']   = ((T[b5] - T[b4])/ (T[b5] + T[b4])).values.clip(-5,5)
    T[f'EVI_{b8.split("_")[1]}']    = (2.5 * (T[b8]  - T[b4] ) / ((T[b8]  + 6.0 * T[b4]  - 7.5 * T[b2]) + 1.0)).values.clip(min=-5,max=5)
    T[f'WDRVI_{b8.split("_")[1]}']  = (((8 * T[b8]) - T[b4])/ ((8* T[b8]) + T[b4])).values.clip(-5,5)
    T[f'ExBlue_{b8.split("_")[1]}']  = ((2 * T[b2]) - (T[b3]+T[b4]))
    T[f'ExGreen_{b8.split("_")[1]}']  = ((2 * T[b3]) - (T[b2]+T[b4]) )
    T[f'NDRE7_{b8.split("_")[1]}']   = ((T[b7] - T[b4])/ (T[b7] + T[b4])).values.clip(-5,5)
    T[f'MTCI_{b8.split("_")[1]}']   = ((T[b8a] - T[b6])/ (T[b7] + T[b6])).values.clip(-5,5)
    T[f'VARI_{b8.split("_")[1]}']   = ((T[b3] - T[b4])/ (T[b3] + T[b4] - T[b2])).values.clip(-5,5)
    T[f'b3b1_{b8.split("_")[1]}']  = (T[b3] - T[b1])/ (T[b3] + T[b1])    # B7  / B3
    T[f'b11b8_{b8.split("_")[1]}']  = (T[b11] - T[b8])/ (T[b11] + T[b8])    # B7  / B3
    T[f'b12b11_{b8.split("_")[1]}']  = (T[b12] - T[b11])/ (T[b12] + T[b11])    # B7  / B3
    T[f'b3b4_{b8.split("_")[1]}']  = (T[b3] - T[b4])/ (T[b3] + T[b4])    # B7  / B3
    T[f'b9b4_{b8.split("_")[1]}']  = (T[b9] - T[b4])/ (T[b9] + T[b4])    # B7  / B3
    T[f'b5b3_{b8.split("_")[1]}']  = (T[b5] - T[b3])/ (T[b5] + T[b3])    # B7  / B3
    T[f'b12b3_{b8.split("_")[1]}']  = (T[b12] - T[b3])/ (T[b12] + T[b3])    # B7  / B3
    
    T[f'b2b1_{b8.split("_")[1]}']  = (T[b2] - T[b1])/ (T[b2] + T[b1])    # B7  / B3
    T[f'b4b1_{b8.split("_")[1]}']  = (T[b4] - T[b1])/ (T[b4] + T[b1])    # B7  / B3
    T[f'b11b3_{b8.split("_")[1]}']  = (T[b11] - T[b3])/ (T[b11] + T[b3]) 
    
  for col in Bcols :
    T[col] = np.sqrt(T[col])
  
  for col in Vcols :
    T[col] = np.sqrt(T[col])
    
  for col1,col2,col3,col4,col5,col6,col7,col8 in zip(Obs1,Obs2,Obs3,Obs4,Obs5,Obs6,Obs7,Obs8) :
    # T[f'{col1}_{col2}'] = T[[col1,col2]].mean(axis=1)
    T[f'{col1.split("_")[0]}_std'] = T[[col1,col2,col3,col4,col5,col6,col7,col8]].std(axis=1)
    T[f'{col1.split("_")[0]}_mean'] = T[[col1,col2,col3,col4,col5,col6,col7,col8]].mean(axis=1)

    
  # NDVI 
  T['water']       = T['NDVI_Month4'].apply(lambda x :1 if x<0 else 0)
  T['dense_green'] = T['NDVI_Month4'].apply(lambda x :1 if x>=0.5 else 0)
  T['not_green']   = T['NDVI_Month4'].apply(lambda x :1 if( (x>0) & (x<0.5)) else 0)

  T['high_green'] = T['SAVI_Month4'].apply(lambda x :1 if x<0.1 else 0)
  T['low_green']  = T['SAVI_Month4'].apply(lambda x :1 if x>=0.8 else 0)
  T['chlorophyll_EVI'] = T['EVI_Month4'].apply(lambda x :1 if( (x>0.2) & (x<0.8)) else 0)

  # process Vegetation indexes

  ObsN   = T.filter(like='NDVI_').columns.tolist()
  ObsSA  = T.filter(like='SAVI_').columns.tolist()
  ObsCC  = T.filter(like='CCCI_').columns.tolist()
  ObsWDR = T.filter(like='WDRVI_').columns.tolist()
  ObsNDRE7 = T.filter(like='NDRE7_').columns.tolist()

  return T

In [ ]:
Train = create_train()
Test = create_test()

In [ ]:
Train.shape , Test.shape

In [ ]:
Train2 = createObs2_train()
Test2 = createObs2_test()

In [ ]:
Train2.shape , Test2.shape

In [ ]:
Train3 = createObs3_train()
Test3 = createObs3_test()

In [ ]:
Train3.shape , Test3.shape

In [ ]:
Train4 = createObs4_train()
Test4 = createObs4_test()

In [ ]:
Train4.shape , Test4.shape

In [ ]:
Train = process(Train)
Test = process(Test)

In [ ]:
Train.shape , Test.shape

In [ ]:
Train2 = process(Train2)
Test2 = process(Test2)

In [ ]:
Train2.shape , Test2.shape

In [ ]:
Train3 = process(Train3)
Test3 = process(Test3)

In [ ]:
Train3.shape , Test3.shape

In [ ]:
Train4 = process(Train4)
Test4 = process(Test4)

In [ ]:
Train4.shape , Test4.shape

In [ ]:
Train = pd.concat([Train,Train2.drop(columns=['field_id','label']),Train3.drop(columns=['field_id','label']),Train4.drop(columns=['field_id','label'])],axis=1)
Train.shape

In [ ]:
Test = pd.concat([Test,Test2.drop(columns=['field_id']),Test3.drop(columns=['field_id'])],axis=1)
Test = pd.merge(Test,Test4,on='field_id',how='left')
Test.shape

# MODELING

In [ ]:
X    = Train.fillna(-999).replace(np.inf,9999).drop(['field_id','label'], axis=1)
y    = Train.label
TEST = Test.fillna(-999).replace(np.inf,9999).drop(['field_id'], axis=1)

In [ ]:
TEST.columns = X.columns.tolist()

In [ ]:
data = pd.concat([X,TEST])
qt=QuantileTransformer(output_distribution="normal",random_state=42)
data= pd.DataFrame(qt.fit_transform(data),columns=X.columns)

In [ ]:
X = data[:X.shape[0]].values
TEST = data[X.shape[0]:].values

In [ ]:
X.shape  , TEST.shape

# Neural Network

In [ ]:
def CUSTOM_ANN_CNN(shape1) :
  # define two sets of inputs
  input_1 = tf.keras.layers.Input(shape = (shape1),name='20_Feats')
  input_2 = tf.keras.layers.Input(shape = (shape1, 1),name='20_Feats_Conv')
  
  # create Convolution Layer 
  head_conv = Convolution1D(filters=128, kernel_size=1, input_shape=(shape1, 1),name='Conv_Layer')(input_2)
  head_conv = Activation('relu',name='Conv_Activation')(head_conv)
  head_conv = Flatten()(head_conv)
  
  # create a simple 2 layers for input_1
  # Layer 1 -- input_1
  head_1 = tf.keras.layers.BatchNormalization()(input_1)
  head_1 = tf.keras.layers.Dense(512, activation = "relu")(head_1)
  head_1 = tf.keras.layers.Dropout(0.15)(head_1)
  # Layer 2 -- input_1
  head_1 = tf.keras.layers.BatchNormalization()(head_1)
  head_1 = tf.keras.layers.Dense(256, activation = "relu")(head_1)
  head_1 = tf.keras.layers.Dropout(0.15)(head_1)
    
  head_1 = tf.keras.layers.BatchNormalization()(head_1)  
  head_1 = tf.keras.layers.add([head_1, Dense(256, activation='relu')(head_1)])
  head_1 = tf.keras.layers.Dropout(0.15)(head_1)

  # Layer 3 -- input_1
  head_1 = tf.keras.layers.BatchNormalization()(head_1)
  head_1 = tf.keras.layers.Dense(128, activation = "relu")(head_1)
  out_1 = tf.keras.layers.Dropout(0.2)(head_1)


  output = tf.keras.layers.Dense(9,activation = "softmax",name='FC_Output')(out_1)
  
  model = tf.keras.models.Model(inputs = [input_1,input_2], outputs = output)
  model.compile(loss='categorical_crossentropy', optimizer='adam')
  
  return model 

In [ ]:
# Function to seed everything

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [ ]:
# reshape test data For Conv Layers Input
X_test_r = np.zeros((len(TEST), X.shape[1], 1))
X_test_r[:, :, 0] = TEST[:, :]

y_train = y.copy()
n_labels = y.unique().shape[0]

# we need to binarize the labels for the neural net

LE = LabelEncoder()
ytrain_enc = pd.get_dummies(y_train).values
TARGETS = pd.get_dummies(y_train).columns

y_oof = np.zeros([X.shape[0], n_labels])
y_test = np.zeros([TEST.shape[0], n_labels])

i = 0
metrics = list()
apply_aug = False

In [ ]:
n_splits = 10
kf = StratifiedKFold(n_splits=n_splits, random_state=47, shuffle=True)

for idx , (tr_idx, val_idx) in enumerate(kf.split(X, y_train)):
    # Verbosity
    VERBOSE = 0

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                                      mode = 'min',
                                                      patience = 25,
                                                      restore_best_weights = True,
                                                      verbose = 1)
    checkpoint = tf.keras.callbacks.ModelCheckpoint(f'Radiant_NN_{idx}.h5',
                                                    monitor = 'val_loss',
                                                    verbose = 1,
                                                    save_best_only = True,
                                                    save_weights_only = True)

    X_tr, X_vl = X[tr_idx, :], X[val_idx, :]
    y_tr, y_vl = ytrain_enc[tr_idx], ytrain_enc[val_idx]
    y_train_, y_vld_ = y_train[tr_idx], y_train[val_idx]

    # reshape train data For Conv Layers Input
    X_train_r = np.zeros((len(X_tr), X.shape[1], 1))
    X_train_r[:, :, 0] = X_tr[:, :]

    # reshape validation data For Conv Layers Input
    X_valid_r = np.zeros((len(X_vl), X.shape[1], 1))
    X_valid_r[:, :, 0] = X_vl[:, :]
    
    # 1 CNN ANN
    seed_everything(seed=1)
    model_cnn_nn = CUSTOM_ANN_CNN(shape1=X.shape[1])
    model_cnn_nn.fit([X_tr,X_train_r], y=y_tr, batch_size=64, epochs=200, verbose=1,
                     validation_data=([X_vl,X_valid_r], y_vl),
                     callbacks = [early_stopping,  checkpoint])
    
    model_cnn_nn = CUSTOM_ANN_CNN(shape1=X.shape[1])
    model_cnn_nn.load_weights(f'Radiant_NN_{idx}.h5')
    y_pred_cnn_nn = model_cnn_nn.predict([X_vl,X_valid_r])
    test_pred_cnn_nn = model_cnn_nn.predict([TEST,X_test_r])
    
    
    y_oof[val_idx, :] = y_pred_cnn_nn
    y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
    metric = log_loss(y_vl, y_pred_cnn_nn)
    print("fold #{} Log Loss: {}".format(i, metric))
    
    i += 1
    test_pred = test_pred_cnn_nn 
    y_test += test_pred / n_splits
    metrics.append(metric)
    
metrics = np.array(metrics).mean()
print(f'Full Log loss {metrics}') 

In [ ]:
print('NN LOG LOSS :',log_loss(y_train,y_oof)) 

In [ ]:
# In this part we format the DataFrame to have column names and order similar to the sample submission file. 
pred_df = pd.DataFrame(y_test)
pred_df = pred_df.rename(columns={
    0:'Crop_ID_1',
    1:'Crop_ID_2', 
    2:'Crop_ID_3',
    3:'Crop_ID_4',
    4:'Crop_ID_5',
    5:'Crop_ID_6',
    6:'Crop_ID_7',
    7:'Crop_ID_8',
    8:'Crop_ID_9'
})
pred_df['field_id'] = Test['field_id'].astype('int').values
pred_df = pred_df[['field_id', 'Crop_ID_1', 'Crop_ID_2', 'Crop_ID_3', 'Crop_ID_4', 'Crop_ID_5', 'Crop_ID_6', 'Crop_ID_7', 'Crop_ID_8', 'Crop_ID_9']]
pred_df.head()

In [ ]:
# Write the predicted probabilites to a csv for submission
pred_df.to_csv('S2_NN.csv', index=False)

In [ ]:
np.save('S2_NN.npy',y_oof)